---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [44]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

In [2]:
pd.options.display.max_columns = None

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
aa = pd.read_csv("City_Zhvi_AllHomes.csv")

In [5]:
dc_states = pd.DataFrame.from_dict(states, orient='index')

In [6]:
bb = pd.read_table("university_towns.txt", header=None,names=["university"])
bb.head(10)

/Users/erdemalpkaya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


,university
0,Alabama[edit]
1,Auburn (Auburn University)[1]
2,Florence (University of North Alabama)
3,Jacksonville (Jacksonville State University)[2]
4,Livingston (University of West Alabama)[2]
5,Montevallo (University of Montevallo)[2]
6,Troy (Troy University)[2]
7,"Tuscaloosa (University of Alabama, Stillman Co..."
8,Tuskegee (Tuskegee University)[5]
9,Alaska[edit]


In [7]:
cc = pd.read_excel("gdplev.xls")
cc.head(23)

,"Current-Dollar and ""Real"" Gross Domestic Product",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,2016-09-29 00:00:00
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Annual,NaN,NaN,NaN,Quarterly,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,(Seasonally adjusted annual rates),NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,NaN,NaN,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1929,104.6,1056.6,NaN,1947q1,243.1,1934.5,NaN
8,1930,92.2,966.7,NaN,1947q2,246.3,1932.3,NaN
9,1931,77.4,904.8,NaN,1947q3,250.1,1930.3,NaN


In [8]:
recession = pd.read_excel("gdplev.xls",skiprows=(range(0,7)))

In [9]:
def get_list_of_university_towns():

    bb = pd.read_table("university_towns.txt", header=None,names=["university"])
    '''
    Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    State= []
    RegionName =[]
    for i, k in enumerate(bb["university"]):
        if(k[-6:]=="[edit]"):
            term = k[:-6]
            term = term.strip()
            
        else:
            State.append(term)
            term2 = bb['university'][i].split("(")[0]
            term2 =term2.strip()
            RegionName.append(term2)
    
    data = {"State":State,"RegionName":RegionName}

    df = pd.DataFrame(data)
    df = df.reindex(columns = ['State','RegionName'])

    return df

In [10]:
get_list_of_university_towns()

/Users/erdemalpkaya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until


,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [11]:
def get_recession_start():
    
    '''
    Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3
    '''
    
    recession = pd.read_excel("gdplev.xls",skiprows=(range(0,219)), usecols=(4,5))
    recession.columns = ['Quarter','GDP']
    
    for i in range(len(recession)-2):
        if(recession['GDP'][i]>recession['GDP'][i+1] and recession['GDP'][i+1]>recession['GDP'][i+2]):
            return recession['Quarter'][i]

In [12]:
get_recession_start()

'2008q3'

In [13]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    recession = pd.read_excel("gdplev.xls",skiprows=(range(0,219)), usecols=(4,5))
    recession.columns = ['Quarter','GDP']
    
    for i in range(len(recession)-2):
        if(recession['GDP'][i]>recession['GDP'][i+1] and recession['GDP'][i+1]>recession['GDP'][i+2] and recession['GDP'][i+2]<recession['GDP'][i+3] and recession['GDP'][i+3]<recession['GDP'][i+4]) :
            return recession['Quarter'][i+4]



In [14]:
get_recession_end()

'2009q4'

In [15]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    recession = pd.read_excel("gdplev.xls",skiprows=(range(0,219)), usecols=(4,5))
    recession.columns = ['Quarter','GDP']
    for i in range(len(recession)-2):
        if(recession['GDP'][i]>recession['GDP'][i+1] and recession['GDP'][i+1]>recession['GDP'][i+2] and recession['GDP'][i+2]<recession['GDP'][i+3] and recession['GDP'][i+3]<recession['GDP'][i+4]) :
            return recession['Quarter'][i+2]
    



In [16]:
get_recession_bottom()

'2009q2'

In [17]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    
    '''
    aa = pd.read_csv("City_Zhvi_AllHomes.csv")
    global states
    aa = aa[aa.columns.drop(list(aa.filter(regex='199')))]
    dc_states = pd.DataFrame.from_dict(states, orient='index')
    
    dc_states.reset_index(inplace=True)
    dc_states.columns = ['State','StateName']
    
    aamain = aa.iloc[:,:6]
    aaquarter = aa.iloc[:,6:]
    
    
    res = (aaquarter.groupby(pd.PeriodIndex(aaquarter.columns, freq='Q'), axis=1)
                  .mean()
                  .rename(columns=lambda c: str(c).lower()))
    mainQ = aamain.join(res)
    mainMerge = mainQ.merge(dc_states,left_on='State', right_on='State', right_index=False)
    mainMerge.set_index(['StateName','RegionName'],inplace=True)
    mainMerge.drop(['RegionID','State','Metro','CountyName','SizeRank'],axis=1, inplace=True)
    #mainMerge.reset_index(inplace=True)

    
    
    
    return mainMerge

In [18]:
convert_housing_data_to_quarters()

2000q1         2000q2  \
StateName RegionName                                              
New York  New York                           NaN            NaN   
          Rochester                 53566.666667   53233.333333   
          Yonkers                  222233.333333  230900.000000   
          Syracuse                  56800.000000   57866.666667   
          Town of Greece            95100.000000   95500.000000   
          Albany                    91566.666667   92100.000000   
          Town of Poughkeepsie     131966.666667  135433.333333   
          New Rochelle             362766.666667  370866.666667   
          Town of Colonie          110600.000000  112400.000000   
          Mount Vernon             239333.333333  247300.000000   
          Utica                     50700.000000   51233.333333   
          Valley Stream            223166.666667  225166.666667   
          Schenectady               72266.666667   73100.000000   
          White Plains             284333.333333  297833.333333   
          Town of Hempstead        170166.666667  172266.666667   
          Town of Binghamton        81633.333333   82500.000000   
          Town of Newburgh         116000.000000  118400.000000   
          Troy                      74733.333333   76533.333333   
          Niagara Falls             50300.000000   50133.333333   
          Irondequoit               83400.000000   83500.000000   
          Binghamton                51766.666667   52000.000000   
          Massapequa               248833.333333  251233.333333   
          Town of Elmira            51200.000000   51600.000000   
          Henrietta                 94566.666667   94400.000000   
          Town of Webster          126433.333333  128600.000000   
          Patchogue                134400.000000  135200.000000   
          Clifton Park             140633.333333  142500.000000   
          Freeport                 183833.333333  186433.333333   
          Lindenhurst              168733.333333  169100.000000   
          Levittown                189966.666667  193366.666667   
...                                          ...            ...   
Vermont   Plainfield                         NaN            NaN   
          West Brattleboro                   NaN            NaN   
          Duxbury                            NaN            NaN   
          Rochester                          NaN            NaN   
          Sharon                             NaN            NaN   
          White River Junction               NaN            NaN   
          Irasburg                           NaN            NaN   
          Mt Holly                           NaN            NaN   
          Ferrisburgh                        NaN            NaN   
          Lincoln                            NaN            NaN   
          Franklin                           NaN            NaN   
          Bridport                           NaN            NaN   
          Fayston                            NaN            NaN   
          Tunbridge                          NaN            NaN   
          Ryegate                            NaN            NaN   
          Shaftsbury                         NaN            NaN   
          Shoreham                           NaN            NaN   
          Fairlee                            NaN            NaN   
          Corinth                            NaN            NaN   
          Brookfield                         NaN            NaN   
          Orwell                             NaN            NaN   
          Town of Wallingford                NaN            NaN   
          Stamford                           NaN            NaN   
          Mendon                             NaN            NaN   
          Morrisville                        NaN            NaN   
          Town of Marshfield                 NaN            NaN   
          Graniteville-East Barre            NaN            NaN   
          Town of Manc

In [110]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    df = convert_housing_data_to_quarters()
    university_towns = get_list_of_university_towns()
    
    df['differ'] = df[get_recession_start()] / df[get_recession_bottom()]
    df['university'] = np.nan
    
    for i,k in enumerate(df.index):
        
        if k[1] in list(university_towns['RegionName']):
            df['university'][i]=True
        else:
            df['university'][i] =False

    df.dropna(subset=['differ'],inplace=True)
    
    university_towns_mean=df['differ'][df['university']==1]
    not_university_mean = df['differ'][df['university']==0]
    t,p = ttest_ind(not_university_mean, university_towns_mean)
    
    mean_result = "university town" if np.mean(university_towns_mean) < np.mean(not_university_mean) else "non-university town"

    p_result = True if p<0.01 else False

    
            
    
    
    return p_result, p, mean_result

In [111]:
run_ttest()

/Users/erdemalpkaya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until


(True, 0.00019452075242283457, 'university town')